In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential 
import pandas as pd
import numpy as np
import pathlib

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy c-extensions failed.
- Try uninstalling and reinstalling numpy.
- If you have already done that, then:
  1. Check that you expected to use Python3.9 from "/usr/bin/python3.9",
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy version "1.17.4" you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how you installed Python
     - how you installed numpy
     - your operating system
     - whether or not you have multiple versions of Python installed
     - if you built from source, your compiler versions and ideally a build log

- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.

Note: this error has many possible causes, so please don't comm

In [ ]:

class Manage_data():
    def __init__(self):
        data_dir='/app/tactile_images/'
        self.data_dir= pathlib.Path(data_dir)

    def find_no_of_images(self, obj_id):
        image_dir = os.path.join(self.data_dir, str(obj_id))
        image_dir= pathlib.Path(image_dir)
        no_of_images= len(list(image_dir.glob('*.jpg')))
        return no_of_images
    
    def parse_function(self, filename, label):
        image_string = tf.io.read_file(filename)
        image_decoded = tf.image.decode_jpeg(image_string, channels=3)
        image_resized = tf.image.resize(image_decoded, [480, 640])  # Adjust size as needed
        return image_resized, label

    
    def load_data(self, data_dir, no_of_samples = 50):
        file_paths = []
        image_paths = []
        y = []
        window_size = 5
        for obj_id in range(no_of_samples):
            no_of_images = self.find_no_of_images(obj_id)
            if no_of_images < 40:
                continue
            
            csv_path = os.path.join(data_dir, str(obj_id),'slip_log.csv')
            label = np.genfromtxt(csv_path, delimiter=',', skip_header=1, usecols=1, dtype=None, encoding=None)
            y.append(label)
            for img_id in range(no_of_images):
                image_path = os.path.join(data_dir, str(obj_id), str(img_id)+ '.jpg')
                file_paths.append(image_path)

        y = np.concatenate(y)
        y = np.array(y)
        file_paths = np.array(file_paths)
        return file_paths, y

    def parse_function_sequential(self, filenames, label):
        images = []
        for filename in filenames.numpy():
            image_string = tf.io.read_file(filename.decode('utf-8'))
            image_decoded = tf.image.decode_jpeg(image_string, channels=3)
            image_resized = tf.image.resize(image_decoded, [480, 640])  # Adjust size as needed
            images.append(image_resized)
        images = tf.stack(images)
        return image_resized, label
    
    def load_sequential_data(self, no_of_samples = 50):
        file_paths = []
        image_paths = []
        y = []
        window_size = 5
        for obj_id in range(no_of_samples):
            no_of_images = self.find_no_of_images(obj_id)
            if no_of_images < 40:
                continue
            
            csv_path = os.path.join(self.data_dir, str(obj_id),'slip_log.csv')
            label = np.genfromtxt(csv_path, delimiter=',', skip_header=1, usecols=1, dtype=None, encoding=None)
            y.append(label)
            for img_id in range(no_of_images):
                image_path = os.path.join(self.data_dir, str(obj_id), str(img_id)+ '.jpg')
        for i in range(0, len(image_path) - 4):  # Ensuring sequences of 5 images
            file_paths.append(image_path[i:i+5])
            
        
        y = np.concatenate(y)
        y = np.array(y)
        y = y[: -4]

        file_paths = np.array(file_paths)
        return file_paths, y
    
    def create_sequential_dataset(self):
        file_paths, labels = self.load_sequential_data(self.data_dir)
                # Create a TensorFlow dataset from the file paths and labels
        dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))

        # Use tf.py_function to apply the parse_function
        def wrapped_parse_function(filenames, label):
            return tf.py_function(func=self.parse_function_sequential, inp=[filenames, label], Tout=[tf.float32, tf.float32])

        dataset = dataset.map(wrapped_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.batch(32)  # Adjust batch size as needed
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    def create_dataset(self):
        file_paths, labels = self.load_data(self.data_dir)
        dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
        dataset = dataset.map(self.parse_function, num_parallel_calls=tf.data.AUTOTUNE)
        batch_size = 32  # Adjust batch size as needed
        dataset = dataset.batch(batch_size)
        self.dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    def split_dataset(self):
        dataset_size = len(self.file_paths)
        train_size = int(0.8 * dataset_size)
        self.train_dataset = self.dataset.take(train_size)
        self.val_dataset = self.dataset.skip(train_size)


'/home/rag-tt/workspace/tactile_images/0/0.jpg'

In [ ]:

class create_network():

    def cnn_lstm1(self):

        # Define CNN model
        cnn_model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(480, 640, 3)),
            MaxPooling2D((2, 2)),
            Conv2D(32, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(32, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten()  # Flatten the spatial dimensions
        ])

        cnn_model.summary()
        # Define LSTM model
        lstm_model = Sequential([
            LSTM(64,input_shape=(5, 144768) ),
            Dense(8, activation='relu'),
            Dense(1, activation='sigmoid'),
        ])

        # Combine CNN and LSTM models
        self.model = Sequential([
            TimeDistributed(cnn_model, input_shape=(480, 640, 3)),  # Apply CNN to each frame in the sequence
            (Reshape((5,144768))),
            lstm_model,
        ])

    def train(self):
        cp = ModelCheckpoint('model/', save_best_only=True)
        self.model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001),metrics=['accuracy'])
        self.model.fit(self.train_dataset, epochs=30)


In [ ]:
manage_data = Manage_data()
network = create_network()
if __name__ == "__main__":
    manage_data.load_sequential_data()